In [39]:
suppressWarnings(suppressPackageStartupMessages({
    library(Seurat)
    library(Signac)
    library(EnsDb.Hsapiens.v86)
    library(dplyr)
    library(ggplot2)        
    library(bedr)
    library(SeuratDisk)
    library(scales)
    library(reshape2)
    library(Hmisc)
    library(tidyr)
    library(tidyverse)
    library(crayon)
    library(SeuratData)
    #library(CelliD)
    library(readr)
}))

# Definitions etc

In [40]:
set.seed(1234)
options(digits=2)
stats <- c()

In [41]:
if(!exists("papermill_run")) {
    # prj_name = "Screen1_66guides"
    # secondary_a_path = "/home/vsevim/prj/1012-ckd/S1/analysis/secondary/"
    # save_seurat_h5 = "YES"
    # n_libs = "4"

    prj_name = "Screen2_101vecs"
    n_libs = "9"
    secondary_a_path = "/home/vsevim/prj/1012-ckd/S2/analysis/secondary/"
    save_seurat_h5 = "YES"
}

In [42]:
custom_theme <- theme(
  plot.title = element_text(size=16, hjust = 0.5), 
  legend.key.size = unit(0.7, "cm"), 
  legend.text = element_text(size = 14))

# Load Seurat files

In [43]:
# Load Seurat objects
#libs = c("Lib_1", "Lib_2", "Lib_3", "Lib_4", "Lib_5", "Lib_6", "Lib_7", "Lib_8", "Lib_9")

libs = paste0("Lib_", 1:as.integer(n_libs))
integrated_h5_path <- paste0(secondary_a_path, "integrated/seurat_objects/integrated_wo_guide_calls")
seurat_list = list()

for(lib in libs) {
    h5_path = paste0(secondary_a_path, lib, "/seurat_objects/", prj_name, "_", lib, ".h5seurat")
    print(h5_path)
    seurat_list[[lib]] = LoadH5Seurat(h5_path, verbose = FALSE)
    #seurat_list[[lib]]$library <- lib
    DefaultAssay(seurat_list[[lib]]) <- "SCT"
}

[1] "/home/vsevim/prj/1012-ckd/S2/analysis/secondary/Lib_1/seurat_objects/Screen2_101vecs_Lib_1.h5seurat"


Validating h5Seurat file



[1] "/home/vsevim/prj/1012-ckd/S2/analysis/secondary/Lib_2/seurat_objects/Screen2_101vecs_Lib_2.h5seurat"


Validating h5Seurat file



[1] "/home/vsevim/prj/1012-ckd/S2/analysis/secondary/Lib_3/seurat_objects/Screen2_101vecs_Lib_3.h5seurat"


...

Validating h5Seurat file



[1] "/home/vsevim/prj/1012-ckd/S2/analysis/secondary/Lib_8/seurat_objects/Screen2_101vecs_Lib_8.h5seurat"


Validating h5Seurat file



[1] "/home/vsevim/prj/1012-ckd/S2/analysis/secondary/Lib_9/seurat_objects/Screen2_101vecs_Lib_9.h5seurat"


Validating h5Seurat file



In [44]:
integration_features <- SelectIntegrationFeatures(object.list = seurat_list)

In [45]:
anchors <- FindIntegrationAnchors(object.list = seurat_list, anchor.features = integration_features, verbose = FALSE)

Warning message in CheckDuplicateCellNames(object.list = object.list):
"Some cell names are duplicated across objects provided. Renaming to enforce unique cell names."


In [46]:
# this command creates an 'integrated' data assay
seurat_combined <- IntegrateData(anchorset = anchors, verbose = FALSE)

# Run SCTransform on the combined

In [47]:
DefaultAssay(seurat_combined) <- "integrated"

In [48]:
seurat_combined <- SCTransform(seurat_combined, vst.flavor = "v2", verbose = FALSE) %>% 
          RunPCA() %>% 
          RunUMAP(dims = 1:40, reduction.name = 'umap.rna', reduction.key = 'rnaUMAP_', verbose=FALSE)

PC_ 1 
Positive:  WFDC2, ERBB4, KCNIP4, MECOM, MUC1, S100A2, KRT19, KRT7, MAL, AC019197.1 
	   AGR2, TMPRSS4, KITLG, CSGALNACT1, FAM155A, SLPI, LCN2, SLC12A1, ACSL4, LINC01503 
	   BST2, PDE1A, MACC1, TACSTD2, ITGB6, ARHGEF38, CYP1B1, SFRP1, CD9, PLCB1 
Negative:  FN1, CRYAB, CDH6, MT1X, APOE, VCAM1, NCKAP5, SOD2, MALAT1, FGB 
	   CTSB, AC013287.1, RHEX, SERPINA1, TRPM3, G0S2, VGF, SERPINE1, HP, NEAT1 
	   GDA, SLC13A3, ABLIM3, GPC6, SLC2A9, MT1G, SERPINE2, NRG3, HMGA2, S100A1 
PC_ 2 
Positive:  FTL, FTH1, RPS12, S100A6, RPS2, LGALS1, MT1E, RPL10, TMSB10, CRYAB 
	   RPL17, RPS18, RPL41, MT2A, RPL26, RPLP1, RPS27A, VIM, RPL12, RPL7A 
	   RPS3A, RPL11, RPS8, TMSB4X, RPS3, EEF1A1, RPS6, KRT19, RPL13, RPL10A 
Negative:  KCNIP4, ERBB4, MECOM, MALAT1, AC079352.1, NEAT1, LINC01320, PLCB1, PDE4D, ADAMTS9-AS2 
	   AC019197.1, FGF14, FAM155A, TRABD2B, LRP1B, LINC02532, PKHD1, LINC00621, NLGN1, PDE1A 
	   PKP4, EFNA5, NCKAP5, LINC01781, SDK1, UBE2D2, AC019117.1, LINC00511, WFDC2, KCNQ1OT1 
PC_ 3 

### Perform clustering

In [49]:
seurat_combined <- FindNeighbors(seurat_combined, reduction = "pca", dims = 1:40)
seurat_combined <- FindClusters(seurat_combined, resolution = 0.5)

Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 46051
Number of edges: 1540868

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8669
Number of communities: 16
Elapsed time: 16 seconds


## Add stats slot

In [50]:
df_stats = NULL
for(i in 1:length(seurat_list)) {
    df_stats = rbind(df_stats, as.data.frame(seurat_list[[i]]@misc$stats))
}

Misc(seurat_combined, slot = 'stats') <- df_stats

### Save H5

In [51]:
SaveH5Seurat(seurat_combined, integrated_h5_path, verbose = FALSE, overwrite = T)

Warning message:
"Overwriting previous file /home/vsevim/prj/1012-ckd/S2/analysis/secondary/integrated/seurat_objects/integrated_wo_guide_calls.h5seurat"
Creating h5Seurat file for version 3.1.5.9900



In [52]:
df_stats

Screen,Library,Cells_recovered,Genes_recovered,Protospacers_in_assay,Max_mito_perc_threshold,Min_mito_perc_threshold,Max_n_RNA_threshold,Min_n_RNA_threshold,Max_n_genes_threshold,Min_n_genes_threshold,Cells_after_QC_filtering,Cells_with_sgRNA_count_less_than_100,Doublets_in_filtered_cells,Total_doublets_before_filtering,Singlets_after_filtering,Singlets_perc_after_filtering
<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<dbl>
Screen2_101vecs,Lib_1,5627,29979,202,27,3,63716,-20142,11034,2000,4765,2845,290,407,4475,0.94
Screen2_101vecs,Lib_2,9597,29397,202,27,3,42848,-25292,13036,2000,5485,3161,427,651,5055,0.92
Screen2_101vecs,Lib_3,12681,30647,202,27,3,57213,-34679,16144,2000,6398,3548,680,1319,5717,0.89
Screen2_101vecs,Lib_4,12632,30625,202,26,3,50879,-30311,14860,2000,6109,3339,774,1572,5332,0.87
Screen2_101vecs,Lib_5,11565,30653,202,26,3,60665,-36591,16665,2000,5723,2977,714,1410,5007,0.87
Screen2_101vecs,Lib_6,10176,28895,202,26,3,33994,-19308,9682,2000,6188,3274,836,1184,5345,0.86
Screen2_101vecs,Lib_7,12699,29718,202,24,3,27215,-15911,9938,2000,5463,3076,534,1539,4927,0.90
Screen2_101vecs,Lib_8,12449,29515,202,24,3,28537,-16287,10080,2000,6134,3371,705,1496,5425,0.88
Screen2_101vecs,Lib_9,5516,29387,202,14,3,37017,-2026,8336,2000,4960,4499,192,246,4768,0.96


In [53]:
nrow(seurat_list[[1]]@meta.data)

[1] 4475

In [54]:
length(Cells(subset(seurat_combined, subset = library == 'Lib_1')))
length(Cells(subset(seurat_combined, subset = (status == 'singlet') & (library == 'Lib_1') )))

[1] 4475

[1] 4475

In [55]:
unique(seurat_combined$status)

[1] "singlet"

In [56]:
unique(seurat_list[[4]]$status)

[1] "singlet"

In [57]:
table(seurat_combined$status)


singlet 
  46051 